In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import *

from sklearn.model_selection import cross_val_score

import pygad

from tqdm import tqdm

In [2]:
X_train = np.load('X_train.npy')
X_test  = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_test  = np.load('y_test.npy')

# Классический генетический алгоритм

Оптимизация гипер параметров с помощью PyGad

## KNN

In [4]:
from sklearn.neighbors import KNeighborsClassifier

In [40]:
num_generations = 20
sol_per_pop     = 10

In [41]:
total_evals = num_generations * sol_per_pop
pbar = tqdm(total=total_evals, desc="GA evaluations")

GA evaluations:   0%|          | 1/200 [00:11<36:29, 11.00s/it]


In [42]:
def fitness_func(ga_instance, solution, solution_idx):
    pbar.update(1)
    
    model = KNeighborsClassifier(
        n_neighbors=int(solution[0]),
        p=int(solution[1]),
        n_jobs=-1
    )

    score = cross_val_score(
        model,
        X_train, 
        y_train, 
        cv=3,
        scoring='f1_macro',
        n_jobs=-1
    ).mean()

    return score

In [43]:
gene_space = [
    {'low': 1, 'high': 100},  # n_neighbours
    {'low': 1, 'high': 10}     # p
]

In [44]:
ga_instance = pygad.GA(
    num_generations=num_generations,
    sol_per_pop=sol_per_pop,
    num_parents_mating=5,
    num_genes=2,
    fitness_func=fitness_func,
    gene_space=gene_space,
    parent_selection_type="rank",
    keep_parents=2,
    mutation_type="random",
    mutation_percent_genes=50,
    random_seed=42
)

In [45]:
ga_instance.run()
# pbar.close()
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print(f"Лучшие гиперпараметры: n_neighbors={int(solution[0])}, p={int(solution[1])}")
print(f"Лучшее значение F1-macro: {solution_fitness:.4f}")

GA evaluations:  98%|█████████▊| 195/200 [2:04:23<02:13, 26.61s/it]  

Лучшие гиперпараметры: n_neighbors=2, p=7
Лучшее значение F1-macro: 0.9943


### Лучшие значения

Лучшие гиперпараметры: n_neighbors=2, p=7

Лучшее значение F1-macro: 0.9943

In [13]:
best_model = KNeighborsClassifier(
        n_neighbors=2,
        p=7,
        n_jobs=-1
    )


## SVC

In [53]:
from sklearn.svm import SVC

In [54]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [55]:
populationa_size = 20
num_generations = 50

In [56]:
kernel_options = ['linear', 'poly', 'rbf', 'sigmoid']
gamma_type_options = ['scale', 'value']

In [57]:
pbar2 = tqdm(total=populationa_size * num_generations, desc="GA iterations")

GA iterations:   0%|          | 1/1000 [02:59<49:54:51, 179.87s/it]


In [58]:
def decode_solution(solution):
    """
    Преобразует вектор решения (фиксированной длины) в словарь гиперпараметров для SVC.
    Ожидается, что решение имеет длину 6:
      0: log10(C) -> C = 10^(solution[0])
      1: kernel_idx (0 до 3)
      2: gamma_type_idx (0 или 1): 0 -> 'scale', 1 -> 'value'
      3: log10(gamma) -> используется если kernel в ['poly', 'rbf', 'sigmoid'] и gamma_type=='value'
      4: degree -> используется только если kernel=='poly'
      5: coef0  -> используется если kernel in ['poly', 'sigmoid']
    """
    params = {
        'C': 10 ** solution[0],
        'kernel': kernel_options[int(solution[1])],
        'gamma': 'scale',  # по умолчанию
        'degree': 3,       # дефолтное для poly
        'coef0': 0.0       # дефолтное для poly и sigmoid
    }
    
    # Определение gamma_type
    gamma_type = gamma_type_options[int(solution[2])]
    
    # Для некоторых ядер может использоваться дополнительный параметр gamma
    if params['kernel'] in ['poly', 'rbf', 'sigmoid'] and gamma_type == 'value':
        params['gamma'] = 10 ** solution[3]
    # Иначе оставляем 'scale'
    
    # Если ядро poly, то берем значение для degree
    if params['kernel'] == 'poly':
        params['degree'] = int(solution[4])
    
    # Если ядро poly или sigmoid, то берем значение для coef0
    if params['kernel'] in ['poly', 'sigmoid']:
        params['coef0'] = solution[5]
    
    return params


In [59]:
def fitness_func(ga_instance, solution, solution_idx):
    pbar2.update(1)

    params = decode_solution(solution)
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('svc',    SVC(**params, random_state=42))
    ])

    
    score = cross_val_score(
        pipeline,
        X_train,
        y_train,
        cv=3,
        scoring='f1_macro',
        n_jobs=-1
    ).mean()

    return score


In [60]:
# Явное перечисление вариантов для целочисленных генов:
gene_space = [
    {'low': -3, 'high': 3},                    # log10(C)
    [0, 1, 2, 3],                             # kernel_idx
    [0, 1],                                   # gamma_type_idx
    {'low': -5, 'high': 2},                    # log10(gamma)
    [2, 3, 4, 5],                             # degree
    {'low': -1.0, 'high': 1.0}                 # coef0
]


In [61]:
ga_instance = pygad.GA(
    num_generations=num_generations,
    sol_per_pop=populationa_size,
    num_parents_mating=5,
    fitness_func=fitness_func,
    num_genes=len(gene_space),
    gene_space=gene_space,
    parent_selection_type="sss",
    keep_parents=2,
    crossover_type="single_point",
    mutation_type="random",
    mutation_percent_genes=20
)

In [ ]:
ga_instance.run()

In [ ]:
best_solution, best_solution_fitness, best_idx = ga_instance.best_solution()
best_params = decode_solution(best_solution)

In [ ]:
print("Лучшая приспособленность (f1_macro):", best_solution_fitness)
print("Лучшие гиперпараметры:")
for k, v in best_params.items():
    print(f"  {k}: {v}")